
# Multivariate Time-Series Anomaly Detection (Compliant Ensemble: IForest + LOF + LSTM-AE)

This notebook implements a compact **ensemble** for multivariate time-series anomaly detection:
- **Isolation Forest (IForest)** — tree-based outlier detection
- **Local Outlier Factor (LOF)** — density-based outlier detection
- **LSTM Autoencoder (LSTM-AE)** — sequence-aware reconstruction anomaly detection

It satisfies the following **requirements**:
- Main entrypoint accepts `input_csv_path` and `output_csv_path`
- Modular classes/functions, full **type hints**, and **docstrings**
- PEP8-friendly structure
- Robust **error handling**
- **Edge cases**: all-normal data, anomalies in training, insufficient data, single-feature datasets, perfect predictions, large datasets (≤ 10,000 rows)
- Output CSV includes: `abnormality_score` + `top_feature_1..top_feature_7` (8 columns)
- Training-period calibration targets: mean < 10, max < 25
- Smooth, stable scores (EWMA); jump diagnostics printed


## Project overview

This notebook implements a multivariate time‑series anomaly detection pipeline based on the requirements defined in the accompanying project specification. The goal is to detect anomalies in sensor time‑series data, assign each row a degree of abnormality score on a 0–100 scale, and identify the most important features contributing to each anomaly. The pipeline follows a modular design: data loading and preprocessing; splitting a normal training window; training unsupervised models (Isolation Forest, Local Outlier Factor and an optional LSTM autoencoder); scoring the full dataset; calibrating scores to the 0–100 scale; smoothing scores to avoid sudden jumps; computing feature attributions (via SHAP when available); and writing results to a CSV with additional columns (`abnormality_score`, `top_feature_1` … `top_feature_7`).


In [26]:

# ========
# Config
# ========

from pathlib import Path

INPUT_CSV = Path("C://Users//Garikipati Karthik//Downloads//81ce1f00-c3f4-4baa-9b57-006fad1875adTEP_Train_Test.csv" )  # change path if needed
OUTPUT_CSV = Path("C://Users//Garikipati Karthik//Downloads//test//Anomaly Detection Test//TEP_output_optimized_NEW_2.csv") # where to save results

# Timestamp column (set to None if absent)
TIMESTAMP_COL: str | None = None  # e.g., "Time"

# Training window indices (inclusive)
TRAIN_START_INDEX: int = 0
TRAIN_END_INDEX: int = 4320

# LSTM settings
USE_LSTM: bool = True
SEQ_LEN: int = 30
LSTM_HIDDEN: int = 64
EPOCHS: int = 12
BATCH_SIZE: int = 256
LEARNING_RATE: float = 1e-3

# Ensemble weights (auto-renormalized if a model is missing)
W_IFOREST: float = 0.45
W_LOF: float = 0.25
W_LSTM: float = 0.30

# Smoothing
EWMA_ALPHA: float = 0.15

# Edge-case thresholds and constraints
MIN_REQUIRED_ROWS: int = 72 * 60  # assume per-minute sampling; adjust if needed
TRAIN_OUTLIER_FRACTION_WARN: float = 0.05  # warn if >5% training points > 25
ALL_NORMAL_P99_MAX: float = 20.0  # clamp p99 to <= 20 for all-normal
EPSILON_NOISE: float = 1e-6       # avoid exact zero scores

TOP_K_ATTRS: int = 7
RANDOM_STATE: int = 42

In [27]:

# =======
# Imports
# =======
from __future__ import annotations

import warnings
warnings.filterwarnings("ignore")

from dataclasses import dataclass
from typing import Tuple, List, Dict, Optional

import numpy as np
import pandas as pd

from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import RobustScaler
from sklearn.decomposition import PCA

# LSTM Autoencoder (TensorFlow/Keras)
try:
    import tensorflow as tf
    from tensorflow.keras import Model
    from tensorflow.keras.layers import Input, LSTM, RepeatVector, TimeDistributed, Dense
    from tensorflow.keras.optimizers import Adam
    TF_AVAILABLE: bool = True
except Exception:
    TF_AVAILABLE = False
    Model = object  # type: ignore
    print("[INFO] TensorFlow not available. Set USE_LSTM=False or install TensorFlow to enable LSTM-AE.")

## Data preprocessing

This section defines helper functions and a `DataProcessor` class to load the input CSV, optionally parse a timestamp column, select numeric features, handle missing values and split the dataset into a normal training window and the full dataset. According to the project spec, all sensor columns should be numeric and missing values should be forward/backward filled.


In [28]:

# =========
# Utilities
# =========
import time

def log(msg: str) -> None:
    """Print a log message with a standard prefix.

    Args:
        msg: The message to print.
    """
    print(f"[LOG] {msg}")

def ewma(x: np.ndarray, alpha: float) -> np.ndarray:
    """Exponentially weighted moving average smoothing.

    Args:
        x: Array of scores.
        alpha: Smoothing factor in [0, 1]. 0 disables smoothing.

    Returns:
        Smoothed array.
    """
    if alpha <= 0:
        return x.astype(float)
    y = np.empty_like(x, dtype=float)
    y[0] = float(x[0])
    for i in range(1, len(x)):
        y[i] = alpha * float(x[i]) + (1.0 - alpha) * y[i - 1]
    return y

def robust_zscores(X: np.ndarray) -> np.ndarray:
    """Compute robust z-scores using median and MAD per feature.

    Args:
        X: 2D array [n_samples, n_features].

    Returns:
        2D array of robust z-scores, same shape as X.
    """
    med = np.median(X, axis=0)
    mad = np.median(np.abs(X - med), axis=0) + 1e-9
    return (X - med) / (1.4826 * mad)

def calibrate_scores_to_0_100(train_scores: np.ndarray,
                              all_scores: np.ndarray) -> Tuple[np.ndarray, Dict[str, float]]:
    """Calibrate raw anomaly scores to 0-100 with training anchors.

    Ensures that training-period scores concentrate below 10 on average
    and keep max below ~25 using percentile-based piecewise mapping.

    Args:
        train_scores: Raw scores for the training window.
        all_scores: Raw scores for all samples.

    Returns:
        (scaled_all_scores, calibration_info)
    """
    train_scores = np.asarray(train_scores, dtype=float)
    all_scores = np.asarray(all_scores, dtype=float)

    tmin = float(np.min(train_scores))
    tmax = float(np.max(train_scores))
    p50 = float(np.percentile(train_scores, 50))
    p95 = float(np.percentile(train_scores, 95))

    def _scale(s: float) -> float:
        """
        Scale a 1D or 2D numpy/pandas structure to [0, 1] using min–max,
        returning the scaled array and guarding against zero ranges.
        """
        s = float(s)
        if s <= p50:
            denom = (p50 - tmin) if (p50 - tmin) != 0 else 1.0
            return 5.0 * (s - tmin) / denom
        elif s <= p95:
            denom = (p95 - p50) if (p95 - p50) != 0 else 1.0
            return 5.0 + 15.0 * (s - p50) / denom
        else:
            denom = (tmax - p95) if (tmax - p95) != 0 else 1.0
            return 20.0 + 5.0 * (s - p95) / denom

    scaled = np.array([_scale(s) for s in all_scores], dtype=float)
    scaled = np.clip(scaled, 0.0, 100.0)
    info = {
        "p50": p50,
        "p95": p95,
        "train_max": tmax,
        "train_min": tmin,
        "train_mean": float(np.mean(train_scores)),
    }
    return scaled, info

In [29]:

# ================
# Data Processing
# ================

@dataclass
class DataProcessor:
    """Load and clean time-series tabular data.

    Attributes:
        timestamp_col: Optional name of the timestamp column.
    """
    timestamp_col: Optional[str] = None

    def load(self, path: Path) -> pd.DataFrame:
        """Load a CSV into a DataFrame.

        Args:
            path: CSV file path.

        Returns:
            Loaded DataFrame.
        """
        log(f"Loading CSV: {path}")
        df = pd.read_csv(path)
        if df.empty:
            raise ValueError("Input CSV has no rows.")
        return df

    def clean(self, df: pd.DataFrame) -> pd.DataFrame:
        """Validate timestamp (if provided) and keep numeric columns.

        Forward/backward fill missing values for numeric columns.

        Args:
            df: Raw DataFrame.

        Returns:
            Cleaned DataFrame.
        """
        if self.timestamp_col and self.timestamp_col in df.columns:
            df[self.timestamp_col] = pd.to_datetime(df[self.timestamp_col],
                                                    errors="coerce")
            df = df.dropna(subset=[self.timestamp_col])

        # Keep only numeric columns (and timestamp if present)
        keep_cols: List[str] = []
        for col in df.columns:
            if col == self.timestamp_col:
                keep_cols.append(col)
            elif pd.api.types.is_numeric_dtype(df[col]):
                keep_cols.append(col)

        if not keep_cols or (len(keep_cols) == 1 and keep_cols[0] == self.timestamp_col):
            raise ValueError("No numeric feature columns found in the dataset.")

        df = df[keep_cols].copy()

        num_cols = [c for c in df.columns if c != self.timestamp_col]
        df[num_cols] = df[num_cols].ffill().bfill().fillna(0.0)

        if self.timestamp_col:
            df = df.drop_duplicates(subset=[self.timestamp_col])

        df = df.reset_index(drop=True)
        return df

    def split_train_full(self, df: pd.DataFrame, train_start: int,
                         train_end: int) -> Tuple[pd.DataFrame, pd.DataFrame]:
        """Split DataFrame into training window and full dataset copies.

        Args:
            df: Clean DataFrame.
            train_start: Start index (inclusive).
            train_end: End index (inclusive).

        Returns:
            (train_df, full_df)
        """
        if train_start < 0 or train_end >= len(df) or train_start > train_end:
            raise ValueError("Invalid training window indices.")
        train = df.iloc[train_start:train_end + 1].copy()
        full = df.copy()
        return train, full

## LSTM autoencoder

An optional LSTM autoencoder can be trained on the normal training window to learn temporal patterns. Reconstruction errors serve as an additional anomaly score. If TensorFlow/Keras is not available the LSTM component will be skipped. You can control this behaviour with the `USE_LSTM` flag in the config.


In [30]:

# ==================
# LSTM Autoencoder
# ==================

def make_sequences(X: np.ndarray, seq_len: int) -> np.ndarray:
    """Create sliding-window sequences for LSTM training/inference.

    Args:
        X: 2D array [n_samples, n_features].
        seq_len: Sequence length in rows.

    Returns:
        3D array [n_sequences, seq_len, n_features].
    """
    N, F = X.shape
    if N < seq_len:
        raise ValueError(f"Not enough rows ({N}) for SEQ_LEN={seq_len}.")
    seqs = np.stack([X[i - seq_len:i, :] for i in range(seq_len, N)], axis=0)
    return seqs

def build_lstm_autoencoder(n_features: int, seq_len: int, hidden: int = 64,
                           lr: float = 1e-3) -> Model:
    """Build a simple LSTM autoencoder for reconstruction.

    Args:
        n_features: Number of features.
        seq_len: Sequence length.
        hidden: Hidden units for encoder/decoder LSTM.
        lr: Learning rate.

    Returns:
        Compiled Keras Model.
    """
    inp = Input(shape=(seq_len, n_features))
    x = LSTM(hidden, return_sequences=False)(inp)
    x = RepeatVector(seq_len)(x)
    x = LSTM(hidden, return_sequences=True)(x)
    out = TimeDistributed(Dense(n_features))(x)
    model = Model(inp, out)
    model.compile(optimizer=Adam(learning_rate=lr), loss="mse")
    return model

def lstm_reconstruction_errors(model: Model, seqs: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    """Compute reconstruction errors from an LSTM autoencoder.

    Args:
        model: Trained LSTM-AE model.
        seqs: Input sequences.

    Returns:
        (mse_per_sequence, mean_abs_error_per_feature_per_sequence)
    """
    recons = model.predict(seqs, verbose=0)
    mse_seq = np.mean((recons - seqs) ** 2, axis=(1, 2))
    feat_abs = np.mean(np.abs(recons - seqs), axis=1)  # shape (N_seq, n_features)
    return mse_seq, feat_abs

## Model training and ensemble

This section defines functions to fit an Isolation Forest, Local Outlier Factor (LOF) and the optional LSTM autoencoder on the training data. Data are first robustly scaled and optionally reduced in dimension using PCA for the LOF. The `score_models` function produces normalized anomaly scores for each model, and `ensemble_scores` combines them using configurable weights. Calibration and smoothing of scores are applied later in the pipeline.


In [31]:

# ==========================
# Model Trainer & Ensemble
# ==========================

@dataclass
class EnsembleModels:
    """Container for trained models and preprocessors."""
    iforest: IsolationForest
    lof: LocalOutlierFactor
    lstm_model: Optional[Model]
    scaler: RobustScaler
    pca: Optional[PCA]

def fit_models(trainX: np.ndarray, use_lstm: bool, seq_len: int, hidden: int,
               epochs: int, batch: int, lr: float) -> EnsembleModels:
    """Fit Isolation Forest, LOF (novelty), and optional LSTM-AE on training data.

    Args:
        trainX: Training array [n_train, n_features].
        use_lstm: Whether to train LSTM-AE.
        seq_len: Sequence length for LSTM-AE.
        hidden: Hidden units for LSTM.
        epochs: Training epochs for LSTM-AE.
        batch: Batch size for LSTM-AE.
        lr: Learning rate for LSTM-AE.

    Returns:
        EnsembleModels with fitted components.
    """
    scaler = RobustScaler()
    Xs = scaler.fit_transform(trainX)

    pca: Optional[PCA] = None
    if trainX.shape[1] > 50:
        pca = PCA(n_components=min(50, trainX.shape[1]))
        Xs_pca = pca.fit_transform(Xs)
    else:
        Xs_pca = Xs

    iforest = IsolationForest(
        n_estimators=256, contamination="auto", bootstrap=False,
        random_state=RANDOM_STATE
    ).fit(Xs)

    lof = LocalOutlierFactor(n_neighbors=35, contamination="auto", novelty=True)
    lof.fit(Xs_pca)

    lstm_model: Optional[Model] = None
    if use_lstm and TF_AVAILABLE:
        seqs = make_sequences(Xs, seq_len)
        lstm_model = build_lstm_autoencoder(trainX.shape[1], seq_len, hidden, lr)
        lstm_model.fit(seqs, seqs, epochs=epochs, batch_size=batch,
                       verbose=0, shuffle=True)
    elif use_lstm and not TF_AVAILABLE:
        log("[WARN] USE_LSTM=True but TensorFlow is not available. Skipping LSTM-AE.")

    return EnsembleModels(iforest=iforest, lof=lof, lstm_model=lstm_model,
                          scaler=scaler, pca=pca)

def score_models(models: EnsembleModels, X: np.ndarray, seq_len: int) -> Dict[str, np.ndarray]:
    """Compute normalized anomaly scores from available models.

    Args:
        models: Trained ensemble models and scalers.
        X: Full array [n_samples, n_features].
        seq_len: Sequence length for LSTM scoring.

    Returns:
        Dict with keys: 'iforest', 'lof', optionally 'lstm', '_lstm_feat_abs'.
    """
    Xs = models.scaler.transform(X)
    Xs_pca = models.pca.transform(Xs) if models.pca is not None else Xs

    # Isolation Forest: higher = more anomalous
    if_scores = -models.iforest.decision_function(Xs)
    if_scores = (if_scores - float(np.min(if_scores))) / (float(np.ptp(if_scores)) + 1e-9)

    # LOF: use score_samples (less negative = more outlier → invert sign)
    lof_s = -models.lof.score_samples(Xs_pca)
    lof_s = (lof_s - float(np.min(lof_s))) / (float(np.ptp(lof_s)) + 1e-9)

    scores: Dict[str, np.ndarray] = {"iforest": if_scores, "lof": lof_s, "lstm": None}

    if models.lstm_model is not None:
        seqs = make_sequences(Xs, seq_len)
        mse_seq, feat_abs = lstm_reconstruction_errors(models.lstm_model, seqs)
        lstm_raw = np.zeros(Xs.shape[0], dtype=float)
        lstm_raw[:seq_len] = float(mse_seq[0])
        lstm_raw[seq_len:] = mse_seq
        lstm_s = (lstm_raw - float(np.min(lstm_raw))) / (float(np.ptp(lstm_raw)) + 1e-9)
        scores["lstm"] = lstm_s
        scores["_lstm_feat_abs"] = feat_abs  # (N - seq_len, n_features)
    return scores

def ensemble_scores(scores: Dict[str, np.ndarray]) -> np.ndarray:
    """Weighted ensemble of available model scores in [0, 1].

    Args:
        scores: Dict of per-model normalized scores.

    Returns:
        1D array of ensemble scores.
    """
    parts: List[np.ndarray] = []
    weights: List[float] = []
    if scores.get("iforest") is not None:
        parts.append(scores["iforest"]); weights.append(W_IFOREST)
    if scores.get("lof") is not None:
        parts.append(scores["lof"]); weights.append(W_LOF)
    if scores.get("lstm") is not None:
        parts.append(scores["lstm"]); weights.append(W_LSTM)

    if not parts:
        raise RuntimeError("No model scores available to ensemble.")

    wsum = sum(weights) if weights else 1.0
    weights = [w / wsum for w in weights] if weights else [1.0]

    ens = np.zeros_like(parts[0], dtype=float)
    for p, w in zip(parts, weights):
        ens += w * p
    return ens

## Feature attribution and SHAP explanations

To understand which sensors contribute most to each detected anomaly, this section computes feature attributions for every row. When the `shap` library is available, SHAP values for the Isolation Forest are computed with `shap.TreeExplainer`. Absolute SHAP values reflect each feature’s contribution to the anomaly score and can be combined with LSTM reconstruction errors. If SHAP is not installed, a fallback method ranks features by their robust z‑scores and, when available, LSTM reconstruction errors. The top `k` features are returned per row.


In [32]:

# ==============================
# Feature Attribution (with SHAP fallback)
# ==============================

def compute_feature_attributions(
    df_num: pd.DataFrame,
    scores: Dict[str, np.ndarray],
    models: EnsembleModels,
    seq_len: int,
    top_k: int,
) -> List[List[str]]:
    """Compute the top-`k` contributing features per row using SHAP when possible.

    This function attempts to explain anomaly scores produced by the ensemble. When
    the SHAP package is available, SHAP values for the Isolation Forest are
    computed via ``shap.TreeExplainer`` on the scaled feature matrix. Absolute
    SHAP values indicate each feature's contribution to the anomaly score. These can
    optionally be blended with per-feature reconstruction errors from the LSTM
    autoencoder. If SHAP is not installed, the function falls back to ranking
    features by their robust z-scores and, when available, LSTM reconstruction
    errors.

    Args:
        df_num: DataFrame of numeric features (no timestamp column).
        scores: Dictionary containing raw anomaly scores and optional ``'_lstm_feat_abs'``.
        models: Container with fitted models and scalers.
        seq_len: Sequence length used for the LSTM autoencoder.
        top_k: Number of top features to return.

    Returns:
        A list of length ``len(df_num)`` where each element is a list of the
        ``top_k`` feature names contributing to that row's anomaly score.
    """
    X = df_num.to_numpy(dtype=float)
    Xs = models.scaler.transform(X)
    n, _ = Xs.shape

    # Try to compute SHAP values using TreeExplainer
    shap_norm = None
    try:
        import shap  # type: ignore
        explainer = shap.TreeExplainer(models.iforest)
        shap_values = explainer.shap_values(Xs)
        shap_abs = np.abs(shap_values)
        shap_norm = shap_abs / (np.max(shap_abs, axis=0, keepdims=True) + 1e-9)
    except Exception:
        shap_norm = None

    # Robust z-scores
    rz = np.abs(robust_zscores(Xs))
    rz_norm = rz / (np.max(rz, axis=0, keepdims=True) + 1e-9)

    # LSTM contributions
    lstm_feat: Optional[np.ndarray] = None
    if "_lstm_feat_abs" in scores:
        lstm_feat = np.zeros_like(rz)
        feat_abs = scores["_lstm_feat_abs"]
        lstm_feat[:seq_len] = feat_abs[0]
        lstm_feat[seq_len:] = feat_abs
        lstm_feat = lstm_feat / (np.max(lstm_feat, axis=0, keepdims=True) + 1e-9)

    if shap_norm is not None:
        if lstm_feat is not None:
            contrib = 0.7 * shap_norm + 0.3 * lstm_feat
        else:
            contrib = shap_norm
    else:
        if lstm_feat is not None:
            contrib = 0.6 * rz_norm + 0.4 * lstm_feat
        else:
            contrib = rz_norm

    cols = df_num.columns.to_numpy()
    topk_names: List[List[str]] = []
    for i in range(n):
        idx_sorted = np.argsort(contrib[i])[::-1][:top_k]
        topk_names.append(cols[idx_sorted].tolist())
    return topk_names


## Full anomaly detection pipeline

The `run_pipeline` function orchestrates the end‑to‑end workflow: it loads and cleans the data, checks that the dataset is large enough (at least 72 hours of per‑minute samples, as recommended in the project spec), splits out the training window, fits the models, scores and calibrates the full dataset, smooths the scores, computes feature attributions, and writes the results to a CSV. It also prints diagnostic information to verify that the training scores meet the success criteria (mean < 10, max < 25) and warns if too many training points exceed the 25 threshold.


In [33]:

# ===============
# Main Pipeline
# ===============
def run_pipeline(input_csv: Path,
                 output_csv: Path,
                 timestamp_col: Optional[str],
                 train_start: int,
                 train_end: int,
                 use_lstm: bool,
                 seq_len: int,
                 hidden: int,
                 epochs: int,
                 batch_size: int,
                 lr: float,
                 ewma_alpha: float,
                 top_k_attrs: int) -> None:
    """Run the full anomaly detection pipeline and write CSV output.

    Args:
        input_csv: Input CSV path.
        output_csv: Output CSV path to write results.
        timestamp_col: Optional timestamp column name.
        train_start: Start index (inclusive) for training window.
        train_end: End index (inclusive) for training window.
        use_lstm: Whether to include LSTM-AE.
        seq_len: Sequence length for LSTM-AE.
        hidden: Hidden units for LSTM.
        epochs: LSTM training epochs.
        batch_size: LSTM batch size.
        lr: LSTM learning rate.
        ewma_alpha: Smoothing factor for scores.
        top_k_attrs: Number of top feature names to output per row.
    """
    t0 = time.time()
    dp = DataProcessor(timestamp_col=timestamp_col)

    # Load and clean
    df_raw = dp.load(input_csv)
    df = dp.clean(df_raw)

    # Edge-case: dataset size requirement
    if len(df) < MIN_REQUIRED_ROWS:
        raise ValueError(
            f"Insufficient data: need at least {MIN_REQUIRED_ROWS} rows (~72 hours at 1-min cadence), "
            f"got {len(df)}."
        )

    # Numeric columns only
    num_cols = [c for c in df.columns if c != timestamp_col]
    if not num_cols:
        raise ValueError("No numeric columns available after cleaning.")

    # Split
    train_df, full_df = dp.split_train_full(df, train_start, train_end)
    train_num = train_df[num_cols].to_numpy(dtype=float)
    full_num = full_df[num_cols].to_numpy(dtype=float)

    log(f"Train rows: {len(train_df)} | Full rows: {len(full_df)} | Features: {len(num_cols)}")


    # Fit models on training only
    models = fit_models(trainX=train_num, use_lstm=use_lstm, seq_len=seq_len,
                        hidden=hidden, epochs=epochs, batch=batch_size, lr=lr)

    # Score on full dataset
    raw_scores = score_models(models, full_num, seq_len=seq_len)
    ens_raw = ensemble_scores(raw_scores)

    # Calibration using training window
    train_mask = np.zeros(len(full_df), dtype=bool)
    train_mask[train_start:train_end + 1] = True
    train_raw = ens_raw[train_mask]
    scaled_0_100, calib_info = calibrate_scores_to_0_100(train_raw, ens_raw)

    # Smooth
    smoothed = ewma(scaled_0_100, ewma_alpha)

    # Edge-case: all-normal clamp and epsilon noise
    p99 = float(np.percentile(smoothed, 99))
    if p99 < ALL_NORMAL_P99_MAX:
        smoothed = np.minimum(smoothed, ALL_NORMAL_P99_MAX)
    smoothed = smoothed + EPSILON_NOISE

    # Jump diagnostics
    jumps = np.abs(np.diff(smoothed, prepend=smoothed[0]))
    max_jump = float(np.max(jumps))
    pct_gt10 = float(100.0 * np.mean(jumps > 10.0))

    log(f"Calibration: {calib_info}")
    log(f"Max consecutive jump: {max_jump:.2f} | % jumps > 10: {pct_gt10:.3f}%")

    # Training stats (success criteria)
    train_scores = smoothed[train_mask]
    mean_train = float(np.mean(train_scores))
    max_train = float(np.max(train_scores))
    print("=== Training Period Checks ===")
    print(f"Mean score: {mean_train:.2f}  (target < 10)")
    print(f"Max score:  {max_train:.2f}  (target < 25)")

    # Warn if too many training outliers (>25)
    frac_train_gt25 = float(np.mean(train_scores > 25.0))
    if frac_train_gt25 > TRAIN_OUTLIER_FRACTION_WARN:
        print("[WARN] More than 5% of training-window points exceed 25. "
              "Training period may contain anomalies.")

    # Feature attributions
    topk = compute_feature_attributions(full_df[num_cols], raw_scores, models,
                                        seq_len=seq_len, top_k=top_k_attrs)

    # Assemble output
    out = df.copy()
    out["abnormality_score"] = smoothed
    for i in range(top_k_attrs):
        out[f"top_feature_{i + 1}"] = [row[i] if i < len(row) else "" for row in topk]

    out.to_csv(output_csv, index=False)
    log(f"Wrote: {output_csv}")

    dt = time.time() - t0
    print(f"Total runtime: {dt:.1f}s")


def main(input_csv_path: str | Path, output_csv_path: str | Path) -> None:
    """Convenience CLI-style entrypoint with only input/output paths.

    Args:
        input_csv_path: Path to input CSV.
        output_csv_path: Path to output CSV to be written.
    """
    run_pipeline(
        input_csv=Path(input_csv_path),
        output_csv=Path(output_csv_path),
        timestamp_col=TIMESTAMP_COL,
        train_start=TRAIN_START_INDEX,
        train_end=TRAIN_END_INDEX,
        use_lstm=USE_LSTM,
        seq_len=SEQ_LEN,
        hidden=LSTM_HIDDEN,
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        lr=LEARNING_RATE,
        ewma_alpha=EWMA_ALPHA,
        top_k_attrs=TOP_K_ATTRS,
    )

## Success criteria and running the pipeline

According to the project requirements, a successful run should produce a training window where the mean abnormality score is below 10 and the maximum score is below 25. If more than 5% of training points exceed 25, the pipeline emits a warning. After running the pipeline you will find the `abnormality_score` and the top contributing features (`top_feature_1`…`top_feature_7`) appended to the input data.

To execute the full pipeline on your own data you can call the `main` function or run the code cell below after updating `INPUT_CSV` and `OUTPUT_CSV` in the config. The results will be saved to the specified output path.


In [34]:

# =====
# Run 
# =====
if __name__ == "__main__":
    # call: main(INPUT_CSV, OUTPUT_CSV)
    run_pipeline(
        input_csv=INPUT_CSV,
        output_csv=OUTPUT_CSV,
        timestamp_col=TIMESTAMP_COL,
        train_start=TRAIN_START_INDEX,
        train_end=TRAIN_END_INDEX,
        use_lstm=USE_LSTM,
        seq_len=SEQ_LEN,
        hidden=LSTM_HIDDEN,
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        lr=LEARNING_RATE,
        ewma_alpha=EWMA_ALPHA,
        top_k_attrs=TOP_K_ATTRS,
    )

[LOG] Loading CSV: C:\Users\Garikipati Karthik\Downloads\81ce1f00-c3f4-4baa-9b57-006fad1875adTEP_Train_Test.csv
[LOG] Train rows: 4321 | Full rows: 26400 | Features: 52
[LOG] Calibration: {'p50': 0.09411971383194555, 'p95': 0.15824427446808256, 'train_max': 0.22820478963849414, 'train_min': 0.002681431514688015, 'train_mean': 0.09772911061730782}
[LOG] Max consecutive jump: 6.76 | % jumps > 10: 0.000%
=== Training Period Checks ===
Mean score: 7.63  (target < 10)
Max score:  23.01  (target < 25)
[LOG] Wrote: C:\Users\Garikipati Karthik\Downloads\test\Anomaly Detection Test\TEP_output_optimized_NEW_2.csv
Total runtime: 79.9s


# Visualization & Reporting Add‑Ons

This section provides analysis with:
1. **Interactive anomaly plots** over time (with severity bands)
2. **Feature attribution heatmaps** (weighted by rank of top contributors)
3. **Automatic anomaly severity reports** (counts, percentages, top features)

> Works directly with generated CSV containing `abnormality_score` and `top_feature_1..7`.
>
> If the notebook already holds a DataFrame with these columns, set `results_df` to it; otherwise, point to the output CSV path.


In [35]:

# === CONFIG / INPUT ===
# If DataFrame with anomaly results is present (including abnormality_score and top_feature_1..7),
# assign it to `results_df` before running this cell. Otherwise, set `OUTPUT_CSV_PATH` to the CSV produced by your pipeline.
from typing import Optional, List, Dict, Tuple
import pandas as pd
import re
import numpy as np

# Try to auto-detect a results CSV; fallback to the known path if present.
# Overide to actual output path.
AUTO_CANDIDATES = [
    # current session (update if necessary)
    "C://Users//Garikipati Karthik//Downloads//test//Anomaly Detection Test//TEP_output_optimized_NEW_2.csv",
]

OUTPUT_CSV_PATH = None
for _p in AUTO_CANDIDATES:
    try:
        import os
        if os.path.exists(_p) and os.path.getsize(_p) > 0:
            OUTPUT_CSV_PATH = _p
            break
    except Exception:
        pass

# Use an existing DataFrame if provided in the notebook; else load from CSV
try:
    results_df  # type: ignore
except NameError:
    results_df = None  # will try to load

if results_df is None:
    assert OUTPUT_CSV_PATH is not None, "Please set OUTPUT_CSV_PATH to your results CSV."
    results_df = pd.read_csv(OUTPUT_CSV_PATH)

required_cols = ["abnormality_score"]
for c in required_cols:
    assert c in results_df.columns, f"Missing required column: {c}"

# Try to find timestamp/datetime column
def detect_time_column(df: pd.DataFrame) -> Optional[str]:
    # Prefer obvious names
    candidates = [c for c in df.columns if c.lower() in ("timestamp", "time", "datetime", "date", "ts")]
    # Otherwise, heuristic: columns containing 'time' or 'date'
    if not candidates:
        candidates = [c for c in df.columns if ("time" in c.lower() or "date" in c.lower())]
    # Validate: convertible to datetime for a subset
    for c in candidates:
        try:
            pd.to_datetime(df[c].head(10), errors="raise")
            return c
        except Exception:
            continue
    return None

TIME_COL = detect_time_column(results_df)
if TIME_COL is not None:
    results_df[TIME_COL] = pd.to_datetime(results_df[TIME_COL], errors="coerce")

# Fill NA abnormality scores with 0
results_df["abnormality_score"] = pd.to_numeric(results_df["abnormality_score"], errors="coerce").fillna(0.0)

# Create severity label
def score_to_severity(s: float) -> str:
    if s <= 10:
        return "0–10 • Normal"
    if s <= 30:
        return "11–30 • Slightly unusual"
    if s <= 60:
        return "31–60 • Moderate"
    if s <= 90:
        return "61–90 • Significant"
    return "91–100 • Severe"

results_df["severity"] = results_df["abnormality_score"].astype(float).apply(score_to_severity)

# Identify top feature columns present
top_feat_cols = [c for c in results_df.columns if re.match(r"^top_feature_[1-7]$", str(c))]


## 1) Interactive anomaly timeline

In [36]:

# === INTERACTIVE ANOMALY TIMELINE ===
# Tries to use Plotly for interactivity; falls back to Matplotlib if Plotly is unavailable.

import math
import numpy as np

# Severity bands
bands = [
    (0, 10, "Normal"),
    (11, 30, "Slightly unusual"),
    (31, 60, "Moderate"),
    (61, 90, "Significant"),
    (91, 100, "Severe"),
]

def plot_interactive_anomaly(df: pd.DataFrame, time_col: Optional[str] = None, score_col: str = "abnormality_score"):
    try:
        import plotly.express as px
        import plotly.graph_objects as go
        has_plotly = True
    except Exception:
        has_plotly = False

    if has_plotly:
        x = df[time_col] if time_col and time_col in df.columns else np.arange(len(df))
        fig = px.line(df, x=x, y=score_col, title="Anomaly Score Over Time")
        # Add severity rectangles
        shapes = []
        for lo, hi, label in bands:
            shapes.append(dict(
                type="rect",
                xref="paper", yref="y",
                x0=0, x1=1, y0=lo, y1=hi,
                fillcolor=None, line=dict(width=0), layer="below"
            ))
        fig.update_layout(shapes=shapes, yaxis_title="Abnormality Score (0–100)",
                          xaxis_title=(time_col if time_col else "Index"))
        # Add markers for high anomalies
        high = df[df[score_col] >= 61]
        if not high.empty:
            fig.add_trace(go.Scatter(
                x=(high[time_col] if time_col and time_col in high.columns else np.arange(len(df))[high.index]),
                y=high[score_col],
                mode="markers",
                name="Significant/Severe",
            ))
        fig.show()
    else:
        # Fallback: Matplotlib
        import matplotlib.pyplot as plt
        x = df[time_col] if time_col and time_col in df.columns else np.arange(len(df))
        plt.figure(figsize=(10, 4.5))
        plt.plot(x, df[score_col])
        plt.title("Anomaly Score Over Time")
        plt.xlabel(time_col if time_col else "Index")
        plt.ylabel("Abnormality Score (0–100)")
        for lo, hi, label in bands:
            plt.axhspan(lo, hi, alpha=0.05)
        # highlight high scores
        high_mask = df[score_col] >= 61
        if high_mask.any():
            plt.scatter(x[high_mask] if not isinstance(x, pd.Series) else x[high_mask].values,
                        df.loc[high_mask, score_col])
        plt.show()

_ = plot_interactive_anomaly(results_df, time_col=TIME_COL, score_col="abnormality_score")


## 2) Feature attribution heatmaps

In [37]:

# === FEATURE ATTRIBUTION HEATMAPS ===
# Build a feature-by-instance matrix from top_feature_1..7, weighted by rank (7 -> top weight to top_feature_1).
# Then plot a heatmap for the top N anomalies.

from collections import Counter, defaultdict

def build_attribution_matrix(df: pd.DataFrame, top_feat_cols: List[str], top_n: int = 200) -> pd.DataFrame:
    if not top_feat_cols:
        raise ValueError("No top_feature_1..7 columns found. Cannot build attribution heatmap.")
    # Select top N most anomalous rows
    sub = df.nlargest(top_n, "abnormality_score").copy()
    # Weighted contribution: top_feature_1 gets weight 7, ... top_feature_7 weight 1
    weights = {f"top_feature_{i}": (8 - i) for i in range(1, 8)}
    # Collect all unique feature names present
    features = set()
    for c in top_feat_cols:
        features.update(sub[c].dropna().astype(str).tolist())
    features = sorted([f for f in features if f and f.lower() != "nan"])
    # Build matrix
    mat = np.zeros((len(features), len(sub)), dtype=float)
    feat_to_idx = {f: i for i, f in enumerate(features)}
    for j, (_, row) in enumerate(sub.iterrows()):
        for c in top_feat_cols:
            f = str(row.get(c, ""))
            if not f or f.lower() == "nan":
                continue
            i = feat_to_idx.get(f)
            if i is not None:
                mat[i, j] += weights.get(c, 1.0)
    # Create DataFrame
    heat_df = pd.DataFrame(mat, index=features, columns=sub.index.astype(str))
    return heat_df, sub

def plot_heatmap(heat_df: pd.DataFrame):
    try:
        import plotly.express as px
        has_plotly = True
    except Exception:
        has_plotly = False

    if has_plotly:
        fig = px.imshow(
            heat_df,
            aspect="auto",
            origin="lower",
            title="Feature Attribution Heatmap (weighted by rank)",
            labels=dict(x="Top Anomaly Instances", y="Features", color="Weight")
        )
        fig.update_yaxes(autorange="reversed")
        fig.show()
    else:
        import matplotlib.pyplot as plt
        plt.figure(figsize=(10, max(6, len(heat_df)//3)))
        plt.imshow(heat_df.values, aspect="auto", origin="lower")
        plt.colorbar(label="Weight")
        plt.yticks(range(len(heat_df.index)), heat_df.index)
        plt.xticks([])
        plt.title("Feature Attribution Heatmap (weighted by rank)")
        plt.show()

heat_df, top_subset = build_attribution_matrix(results_df, top_feat_cols, top_n=200)
plot_heatmap(heat_df)


## 3) Automatic anomaly severity reports

In [38]:

# === AUTOMATIC ANOMALY SEVERITY REPORT ===

from collections import Counter
import json

def severity_summary(df: pd.DataFrame) -> Dict[str, Dict[str, float]]:
    total = len(df)
    counts = df["severity"].value_counts().to_dict()
    perc = {k: (v * 100.0 / total if total else 0.0) for k, v in counts.items()}
    return {"counts": counts, "percentages": perc, "total_rows": total}

def top_features_overall(df: pd.DataFrame, top_feat_cols: List[str], k: int = 15) -> List[Tuple[str, int]]:
    cnt = Counter()
    for c in top_feat_cols:
        cnt.update(df[c].dropna().astype(str).tolist())
    # remove empty/'nan'
    clean = Counter({f:v for f,v in cnt.items() if f and f.lower() != "nan"})
    return clean.most_common(k)

def top_features_by_severity(df: pd.DataFrame, top_feat_cols: List[str], k: int = 10) -> Dict[str, List[Tuple[str, int]]]:
    out = {}
    for sev, grp in df.groupby("severity"):
        out[sev] = top_features_overall(grp, top_feat_cols, k=k)
    return out

report = severity_summary(results_df)
overall_top = top_features_overall(results_df, top_feat_cols, k=20)
by_sev = top_features_by_severity(results_df, top_feat_cols, k=10)

print("=== Anomaly Severity Summary ===")
print(json.dumps(report, indent=2))
print("\n=== Overall Top Contributing Features ===")
for f, c in overall_top:
    print(f"{f}: {c}")
print("\n=== Top Contributing Features by Severity ===")
for sev, items in by_sev.items():
    print(f"\n{sev}")
    for f, c in items:
        print(f"  {f}: {c}")

# Export as CSV/JSON artifacts for your report
out_dir = "anomaly_reports"
import os
os.makedirs(out_dir, exist_ok=True)

pd.DataFrame({"severity": list(report["counts"].keys()), "count": list(report["counts"].values())}).to_csv(
    os.path.join(out_dir, "severity_counts.csv"), index=False
)

pd.DataFrame(overall_top, columns=["feature", "count"]).to_csv(
    os.path.join(out_dir, "top_features_overall.csv"), index=False
)

# Breakout per severity
for sev, items in by_sev.items():
    safe = sev.replace("•", "-").replace(" ", "_").replace("–", "-")
    pd.DataFrame(items, columns=["feature", "count"]).to_csv(
        os.path.join(out_dir, f"top_features_{safe}.csv"), index=False
    )

with open(os.path.join(out_dir, "summary.json"), "w") as f:
    json.dump({"summary": report, "overall_top": overall_top, "by_severity": by_sev}, f, indent=2)

print("\nArtifacts saved in ./anomaly_reports/")


=== Anomaly Severity Summary ===
{
  "counts": {
    "0\u201310 \u2022 Normal": 12497,
    "11\u201330 \u2022 Slightly unusual": 9192,
    "31\u201360 \u2022 Moderate": 3630,
    "61\u201390 \u2022 Significant": 1081
  },
  "percentages": {
    "0\u201310 \u2022 Normal": 47.33712121212121,
    "11\u201330 \u2022 Slightly unusual": 34.81818181818182,
    "31\u201360 \u2022 Moderate": 13.75,
    "61\u201390 \u2022 Significant": 4.09469696969697
  },
  "total_rows": 26400
}

=== Overall Top Contributing Features ===
StripperLevel: 15077
StripperLiquidProductFlowStream11: 13327
ProductSepLevel: 11158
SeparatorPotLiquidFlowStream10: 9450
StripperSteamFlowkgperhr: 4872
StripperUnderflowStream11: 4859
ComponentD11: 4857
ProdSepUnderflowStream10: 4100
ComponentG9: 4093
ReactorTemperatureDegC: 3825
RecycleFlowStream8: 3806
PurgeValveStream9: 3692
PurgeRateStream9: 3618
ComponentC9: 3498
ProductSepTempDegC: 3449
ComponentE6: 3438
TotalFeedStream4: 3341
EFeedStream3: 3270
AFeedStream1: 3269
Compo

## Quick severity bar chart

In [39]:

# === OPTIONAL: Visual bar chart of severity counts ===
import pandas as pd

sev_counts = results_df["severity"].value_counts().reset_index()
sev_counts.columns = ["severity", "count"]

try:
    import plotly.express as px
    fig = px.bar(sev_counts, x="severity", y="count", title="Anomaly Severity Distribution")
    fig.show()
except Exception:
    import matplotlib.pyplot as plt
    plt.figure(figsize=(8,4))
    plt.bar(sev_counts["severity"], sev_counts["count"])
    plt.title("Anomaly Severity Distribution")
    plt.xlabel("Severity")
    plt.ylabel("Count")
    plt.xticks(rotation=45, ha="right")
    plt.tight_layout()
    plt.show()
